In [1]:
import requests
import json
import time
import os
import pandas as pd
import requests
import csv

from global_functions import *

### Information

Input files:
data_processing/bugs_info.csv

Output files:
raw_data/bug_history.csv (the initial bug history)
raw_data/bug_history.csv (bug history after injecting assignment statuses )

In [2]:
bug_info_file = 'data_processing'+os.sep+'bugs_info.csv'

bug_history_file_orig = './raw_data/bug_history.csv'

In [3]:
df_bugs = pd.read_csv(bug_info_file,index_col=False)
df_bugs = trans_to_datetime(df_bugs,['creation_time'])
df_bugs.head()

,id,Product,version,resolution,status,severity,creation_time,priority,release
0,475361,Platform,4.5,FIXED,RESOLVED,normal,2015-08-19 10:50:25,P3,4.5
1,475365,Platform,4.6,FIXED,RESOLVED,minor,2015-08-19 11:34:37,P3,4.6
2,475370,Platform,4.5,DUPLICATE,CLOSED,minor,2015-08-19 12:09:06,P3,4.5
3,475379,Platform,4.5,NaN,NEW,normal,2015-08-19 13:39:12,P3,4.5
4,475407,Platform,4.5,FIXED,RESOLVED,normal,2015-08-19 17:06:10,P3,4.5


In [4]:
len(df_bugs.id.unique())

141010

In [ ]:
def parse_eclipse_bug_history(bug_id):
    tfile = './raw_data/json_history/'+str(bug_id)+'.json'
    if os.path.exists(tfile):
        return
    link = 'https://bugs.eclipse.org/bugs/rest/bug/'+str(bug_id)+'/history'
    ff = requests.get(link)
    plat_bugs = ff.text
    #print(plat_bugs)

    jsonobj = json.loads(plat_bugs)
    jsonobj = jsonobj['bugs']
    with open(tfile, 'w') as outfile:
        json.dump(jsonobj, outfile)

for x in df_bugs.id.unique().tolist():
    parse_eclipse_bug_history(x)

In [ ]:
#get data from json files
def getData(output_csv):
    df = pd.DataFrame()
    for item in output_csv:  
        tdata = pd.io.json.json_normalize(item['history'])
        tdata2 = flatten_nested_data(tdata)
        tdata2['id'] = item['id']
        df = df.append(tdata2)
    return df

In [ ]:
#get data from json files
path_to_json='./raw_data/json_history'

if os.path.exists(bug_history_file_orig):
    bug_history = pd.read_csv(bug_history_file_orig,index_col=False)
else:
    bug_history = pd.DataFrame()

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

print('size to parse:'+str(len(json_files)))
if len(bug_history.columns)>0:
    finished = bug_history.id.unique().tolist()
    finished = [str(x)+'.json' for x in finished]
    json_files = [x for x in json_files if x not in finished]
    
print('size to parse (after filtering parsed jsons):'+str(len(json_files)))

if len(json_files)>0:
    for file in json_files:
        with open(path_to_json+os.sep+file) as f:
            data = json.load(f)
            tmp = getData(data)
            tmp = tmp.rename(index=str,columns={'changes.added':'added',
                                       'changes.field_name':'what',
                                       'changes.removed':'removed'})

            tmp = tmp[['added', 'id', 'removed', 'what', 'when','who']]
            bug_history = bug_history.append(tmp,ignore_index=True)
    #export the complete bug history file
    bug_history.to_csv(bug_history_file_orig,index=False)